# Pre-Processing: Vom Text zu Daten

## Ablauf...

* [Run code](#1-bullet)
* [Tabelle mit Texten importieren](#2-bullet)
* [Den Text "reinigen" und Stop-Wörter entfernen](#3-bullet)
* [Lemmatisierung von Wörtern](#4-bullet)
* [Part of Speech Tagging](#5-bullet)
* [Named Emtities Recognition](6#-bullet)
* [Gewünschten Text exportieren](#7-bullet)

## Run code <a class="anchor" id="1-bullet"></a>

Um Code in *Jupyter Notebooks* auszühren, müssen Sie auf **Run** klicken, das sich in der oberen Leiste finden. Klicken Sie dafür zunächst auf das Feld, das Sie ausführen möchten und anschließend auf **Run**. Sie können auch 'Shift' und 'enter' verwenden. Wenn der Code gerade ausgeführt, erscheint links im Kästchen ein Sternchen. Dies bedeutet, dass das Programm gerade arbeitet. Wurde der Code erfolgreich ausgeführt. erscheint eine Nummer im selben Kästchen.

In [1]:
#languange processing imports
import gensim, spacy, logging, warnings
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
import sys
import re, numpy as np, pandas as pd
from pprint import pprint
import spacy
from openpyxl import Workbook

#For the text overview
from nltk import FreqDist

#Word Cloud and Visualization

from wordcloud import WordCloud, STOPWORDS
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.ticker as mtick
import matplotlib.pyplot as plt

C:\Users\c62255\AppData\Roaming\Python\Python37\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


## Tabelle mit Texten importieren <a class="anchor" id="2-bullet"></a>

Hier sehen Sie die importierte Excel-Tabelle

In [2]:
df = pd.read_excel(('Data/test.xlsx'), engine='openpyxl',)
df_text = df['text']
df.style.set_caption('Excel Tabelle').hide_index()

jahrgang,zeitung,datum,ausgabe,text
1946,Dolomiten,1946-11-10 00:00:00,2,"Achtung! Südtiroler! Die Optanten für Deutschland, die effektiv abgewandert waren und in der Zwischenzeit wieder nach Italien zurückgekehrt sind, werden ersucht, ihre Erklärungen zwecks Wiedererlangung der italienischen Staatsbürger- schaft erst nach dem 20. Februar 1948 abzu geben, um das Optionsamt im gegenwärtigen Zeitpunkt nicht zu überlasten. Die eingebürgerten Optanten, die in Italien verblieben sind, sowie alle diejenigen, welche nach Art. 15 des Optionsdekretes nicht als „abgewandert“ anzusehen sind, werden dringend aufgefordert, ihre Erklärungen nach Art. 2 des erwähnten Optionsdekretes rechtzeitig. d. i. bis zum 20, Februar 1948, einzureichen, und zwar auch dann, wenn sie in den Wählerlisten eingetragen sind, weil sie sonst Gefahr laufen, aus denselben gestrichen zu werden. Es wird ausdrücklich darauf aufmerksam gemacht, daß viele eingebürgerte Optanten die auf Grund der gesetzlichen Bestimmungen nicht als abgewandert zu gelten haben, sowie auch solche, die während der ganzen Zeit Italien überhaupt nie verlassen haben, in den Refistern der Gemeinden als abgewandert figurieren; diese werden daher besonders gebeten, ihre Erklärunden fristgerecht, d. i. bis zum 20. Februar 1948, abzugeben, und falls die Eintragung in die Wählerlisten verweigert werden sollte, umgehend bei der Bezirkskommission (Commissione Elettorale Mandamentale) Einspruch zu erheben. Zu diesem Zwecke ist es erforderlich, daß jeder eingebürgerte Optant, der die obenerwähnte Erklärung abgegeben hat, sich persönlich davon überzeugt, ob die Eintragung in die Wählerlisten erfolgt ist. Die sogenannten Rückwähler, das sind diejenigen, welche vor dem 9. September 1943 bei der Präfektur um Annullierung der Option für Deutschland angesucht haben, müssen sich persönlich davon überzeugen, ob sie auf Grund der Rückoption auch tatsächlich wieder als italienische Staatsbürger eingetragen wurden. Ist dies nicht der Fall oder besieht diesbezüglich irgend eine Unsicherheit, so müssen die selben die im Optionsgesetz vorgesehene Erklärung abdeben, weil sonst die Gelahr besteht, daß sie die italienische Staatsbürgerschaft nicht erlangen. Die Optanten werden aufmerksam gemacht, daß die Präfektur Bozen an alle Gemeinden die Weisung ergehen ließ, bei der Ausgabe der Erklärungsformulare keinerlei Schwierigkeiten zu bereiten, weshalb alle Optanten denen die Ausfolgung der Formulare verweigert wurde ersucht werden, dieselben neuerlich anzufordern."
1947,Dolomiten,1947-11-10 00:00:00,3,"Dabei muß klargestellt werden, daß der jetzigen Belegschaft aus der Wiederanstellung der Südtiroler arbeits- und anstellungsmäßig nicht der geringste Nachteil erwachsen würde. Man sagt nicht etwa, daß die italienischen Angestellten und Arbeiter, welche im Jahre 1940 die Nachfolge der entlassenen Südtiroler antraten, diesen wieder Platz machen müßten. Nein, da die Etschwerke infolge ihrer wachsenden Aufgaben, neue Arbeitskräfte nötig haben, sollten zu den bisherigen die Südtiroler Rückoptanten zusätzlich aufgenommen werden. Es sei auch noch daran erinnert, daß diese Rückoptanten bei den Etschwerken nicht freiwillig aus dem Dienst schieden, sondern nach ihrer Option im Jahre 1939 vom Werk entlassen wurden mit der Begründung, daß die Etschwerke eine öffentliche Körperschaft seien und deswegen nichtitalienische Staatsbürger nicht mehr beschäftigen dürften. Damit fällt auch das letzte scheinheilig Mäntelchen, mit welchem die Wortführer der Belegschaft ihre nationalgehässige Streikdrohung zu umgeben versuchten, nämlich die Behauptung, daß die Etschwerke keine öffentliche Körperschaft seien, auf welche die Be¬ stimmungen des Gesetzes vom Jahre 1952 Anwendung finden."


## Den Text "reinigen" und Stop-Wörter entfernen <a class="anchor" id="3-bullet"></a>

Nun wird der Text gereinigt, das heißt, es werden Satzzeichen entfernt, alle Wörter werden einheitlich klein geschrieben und Stop-Wörter werden entfernt. Stop-Wörter sind Wörter, die häufig vorkommen und haben für gewöhnlich keine Relevanz für weitere Analysen. Hierzu gehören Wörter wie "und", "auf", "mit" usw. 

In [3]:
#Functions to clean, tokenize, and stem the data
import nltk
nltk.download('stopwords')
nltk.download('punkt')

def initial_clean(text):
    text = re.sub(r'[^\w\s]','',text)
    text = text.lower() 
    text = nltk.word_tokenize(text)
    return text

stop_words = stopwords.words('german')#change the language here

stop_words.extend([]) #add additional words here
def remove_stop_words(text):
    return [word for word in text if word not in stop_words]

def apply_all(text):
    return remove_stop_words(initial_clean(text))
df['tokenized'] = df['text'].apply(apply_all) 
df.style.set_caption('Text und Tokens').hide_index()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\c62255\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\c62255\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


jahrgang,zeitung,datum,ausgabe,text,tokenized
1946,Dolomiten,1946-11-10 00:00:00,2,"Achtung! Südtiroler! Die Optanten für Deutschland, die effektiv abgewandert waren und in der Zwischenzeit wieder nach Italien zurückgekehrt sind, werden ersucht, ihre Erklärungen zwecks Wiedererlangung der italienischen Staatsbürger- schaft erst nach dem 20. Februar 1948 abzu geben, um das Optionsamt im gegenwärtigen Zeitpunkt nicht zu überlasten. Die eingebürgerten Optanten, die in Italien verblieben sind, sowie alle diejenigen, welche nach Art. 15 des Optionsdekretes nicht als „abgewandert“ anzusehen sind, werden dringend aufgefordert, ihre Erklärungen nach Art. 2 des erwähnten Optionsdekretes rechtzeitig. d. i. bis zum 20, Februar 1948, einzureichen, und zwar auch dann, wenn sie in den Wählerlisten eingetragen sind, weil sie sonst Gefahr laufen, aus denselben gestrichen zu werden. Es wird ausdrücklich darauf aufmerksam gemacht, daß viele eingebürgerte Optanten die auf Grund der gesetzlichen Bestimmungen nicht als abgewandert zu gelten haben, sowie auch solche, die während der ganzen Zeit Italien überhaupt nie verlassen haben, in den Refistern der Gemeinden als abgewandert figurieren; diese werden daher besonders gebeten, ihre Erklärunden fristgerecht, d. i. bis zum 20. Februar 1948, abzugeben, und falls die Eintragung in die Wählerlisten verweigert werden sollte, umgehend bei der Bezirkskommission (Commissione Elettorale Mandamentale) Einspruch zu erheben. Zu diesem Zwecke ist es erforderlich, daß jeder eingebürgerte Optant, der die obenerwähnte Erklärung abgegeben hat, sich persönlich davon überzeugt, ob die Eintragung in die Wählerlisten erfolgt ist. Die sogenannten Rückwähler, das sind diejenigen, welche vor dem 9. September 1943 bei der Präfektur um Annullierung der Option für Deutschland angesucht haben, müssen sich persönlich davon überzeugen, ob sie auf Grund der Rückoption auch tatsächlich wieder als italienische Staatsbürger eingetragen wurden. Ist dies nicht der Fall oder besieht diesbezüglich irgend eine Unsicherheit, so müssen die selben die im Optionsgesetz vorgesehene Erklärung abdeben, weil sonst die Gelahr besteht, daß sie die italienische Staatsbürgerschaft nicht erlangen. Die Optanten werden aufmerksam gemacht, daß die Präfektur Bozen an alle Gemeinden die Weisung ergehen ließ, bei der Ausgabe der Erklärungsformulare keinerlei Schwierigkeiten zu bereiten, weshalb alle Optanten denen die Ausfolgung der Formulare verweigert wurde ersucht werden, dieselben neuerlich anzufordern.","['achtung', 'südtiroler', 'optanten', 'deutschland', 'effektiv', 'abgewandert', 'zwischenzeit', 'italien', 'zurückgekehrt', 'ersucht', 'erklärungen', 'zwecks', 'wiedererlangung', 'italienischen', 'staatsbürger', 'schaft', 'erst', '20', 'februar', '1948', 'abzu', 'geben', 'optionsamt', 'gegenwärtigen', 'zeitpunkt', 'überlasten', 'eingebürgerten', 'optanten', 'italien', 'verblieben', 'sowie', 'diejenigen', 'art', '15', 'optionsdekretes', 'abgewandert', 'anzusehen', 'dringend', 'aufgefordert', 'erklärungen', 'art', '2', 'erwähnten', 'optionsdekretes', 'rechtzeitig', 'd', 'i', '20', 'februar', '1948', 'einzureichen', 'wählerlisten', 'eingetragen', 'gefahr', 'laufen', 'gestrichen', 'ausdrücklich', 'darauf', 'aufmerksam', 'gemacht', 'viele', 'eingebürgerte', 'optanten', 'grund', 'gesetzlichen', 'bestimmungen', 'abgewandert', 'gelten', 'sowie', 'ganzen', 'zeit', 'italien', 'überhaupt', 'nie', 'verlassen', 'refistern', 'gemeinden', 'abgewandert', 'figurieren', 'daher', 'besonders', 'gebeten', 'erklärunden', 'fristgerecht', 'd', 'i', '20', 'februar', '1948', 'abzugeben', 'falls', 'eintragung', 'wählerlisten', 'verweigert', 'umgehend', 'bezirkskommission', 'commissione', 'elettorale', 'mandamentale', 'einspruch', 'erheben', 'zwecke', 'erforderlich', 'eingebürgerte', 'optant', 'obenerwähnte', 'erklärung', 'abgegeben', 'persönlich', 'davon', 'überzeugt', 'eintragung', 'wählerlisten', 'erfolgt', 'sogenannten', 'rückwähler', 'diejenigen', '9', 'september', '1943',

## Lemmatisierung von Wörtern <a class="anchor" id="4-bullet"></a>
Die lexikographische Reduktion der Flexionsformen eines Wortes auf eine Grundform, also die Festlegung der Grundform eines Lexems und die Anordnung der Lemmata wird auch Lemmatisierung genannt.

In [4]:
from gensim.utils import simple_preprocess
def lemmatization(texts):
    texts_out = []
    nlp = spacy.load('de_core_news_sm')
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc])   
    return texts_out

df['lemmatized'] = lemmatization(df['tokenized'])
df.style.set_caption('Text, tokens und lemmas').hide_index()

jahrgang,zeitung,datum,ausgabe,text,tokenized,lemmatized
1946,Dolomiten,1946-11-10 00:00:00,2,"Achtung! Südtiroler! Die Optanten für Deutschland, die effektiv abgewandert waren und in der Zwischenzeit wieder nach Italien zurückgekehrt sind, werden ersucht, ihre Erklärungen zwecks Wiedererlangung der italienischen Staatsbürger- schaft erst nach dem 20. Februar 1948 abzu geben, um das Optionsamt im gegenwärtigen Zeitpunkt nicht zu überlasten. Die eingebürgerten Optanten, die in Italien verblieben sind, sowie alle diejenigen, welche nach Art. 15 des Optionsdekretes nicht als „abgewandert“ anzusehen sind, werden dringend aufgefordert, ihre Erklärungen nach Art. 2 des erwähnten Optionsdekretes rechtzeitig. d. i. bis zum 20, Februar 1948, einzureichen, und zwar auch dann, wenn sie in den Wählerlisten eingetragen sind, weil sie sonst Gefahr laufen, aus denselben gestrichen zu werden. Es wird ausdrücklich darauf aufmerksam gemacht, daß viele eingebürgerte Optanten die auf Grund der gesetzlichen Bestimmungen nicht als abgewandert zu gelten haben, sowie auch solche, die während der ganzen Zeit Italien überhaupt nie verlassen haben, in den Refistern der Gemeinden als abgewandert figurieren; diese werden daher besonders gebeten, ihre Erklärunden fristgerecht, d. i. bis zum 20. Februar 1948, abzugeben, und falls die Eintragung in die Wählerlisten verweigert werden sollte, umgehend bei der Bezirkskommission (Commissione Elettorale Mandamentale) Einspruch zu erheben. Zu diesem Zwecke ist es erforderlich, daß jeder eingebürgerte Optant, der die obenerwähnte Erklärung abgegeben hat, sich persönlich davon überzeugt, ob die Eintragung in die Wählerlisten erfolgt ist. Die sogenannten Rückwähler, das sind diejenigen, welche vor dem 9. September 1943 bei der Präfektur um Annullierung der Option für Deutschland angesucht haben, müssen sich persönlich davon überzeugen, ob sie auf Grund der Rückoption auch tatsächlich wieder als italienische Staatsbürger eingetragen wurden. Ist dies nicht der Fall oder besieht diesbezüglich irgend eine Unsicherheit, so müssen die selben die im Optionsgesetz vorgesehene Erklärung abdeben, weil sonst die Gelahr besteht, daß sie die italienische Staatsbürgerschaft nicht erlangen. Die Optanten werden aufmerksam gemacht, daß die Präfektur Bozen an alle Gemeinden die Weisung ergehen ließ, bei der Ausgabe der Erklärungsformulare keinerlei Schwierigkeiten zu bereiten, weshalb alle Optanten denen die Ausfolgung der Formulare verweigert wurde ersucht werden, dieselben neuerlich anzufordern.","['achtung', 'südtiroler', 'optanten', 'deutschland', 'effektiv', 'abgewandert', 'zwischenzeit', 'italien', 'zurückgekehrt', 'ersucht', 'erklärungen', 'zwecks', 'wiedererlangung', 'italienischen', 'staatsbürger', 'schaft', 'erst', '20', 'februar', '1948', 'abzu', 'geben', 'optionsamt', 'gegenwärtigen', 'zeitpunkt', 'überlasten', 'eingebürgerten', 'optanten', 'italien', 'verblieben', 'sowie', 'diejenigen', 'art', '15', 'optionsdekretes', 'abgewandert', 'anzusehen', 'dringend', 'aufgefordert', 'erklärungen', 'art', '2', 'erwähnten', 'optionsdekretes', 'rechtzeitig', 'd', 'i', '20', 'februar', '1948', 'einzureichen', 'wählerlisten', 'eingetragen', 'gefahr', 'laufen', 'gestrichen', 'ausdrücklich', 'darauf', 'aufmerksam', 'gemacht', 'viele', 'eingebürgerte', 'optanten', 'grund', 'gesetzlichen', 'bestimmungen', 'abgewandert', 'gelten', 'sowie', 'ganzen', 'zeit', 'italien', 'überhaupt', 'nie', 'verlassen', 'refistern', 'gemeinden', 'abgewandert', 'figurieren', 'daher', 'besonders', 'gebeten', 'erklärunden', 'fristgerecht', 'd', 'i', '20', 'februar', '1948', 'abzugeben', 'falls', 'eintragung', 'wählerlisten', 'verweigert', 'umgehend', 'bezirkskommission', 'commissione', 'elettorale', 'mandamentale', 'einspruch', 'erheben', 'zwecke', 'erforderlich', 'eingebürgerte', 'optant', 'obenerwähnte', 'erklärung', 'abgegeben', 'persönlich', 'davon', 'überzeugt', 'eintragung', 'wählerlisten', 'erfolgt', 'sogenannten', 'rückwähler', 'diejenigen', '9', 'septembe

## Part of Speech Tagging <a class="anchor" id="5-bullet"></a>

Unter **Part-of-speech-Tagging** (POS-Tagging) versteht man die Zuordnung von Wörtern und Satzzeichen eines Textes zu Wortarten (englisch part of speech). Hierzu wird sowohl die Definition des Wortes als auch der Kontext (z. B. angrenzende Adjektive oder Nomen) berücksichtigt.

In [5]:
def tagging(texts, allowed_postags=['NOUN']): # possible tags'NOUN', 'ADJ', 'ADV', 'VERB'
    texts_out = []
    nlp = spacy.load('de_core_news_sm')
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])   
    return texts_out

df['tagging'] = tagging(df['lemmatized'])
df.style.set_caption('Only nouns').hide_index()

jahrgang,zeitung,datum,ausgabe,text,tokenized,lemmatized,tagging
1946,Dolomiten,1946-11-10 00:00:00,2,"Achtung! Südtiroler! Die Optanten für Deutschland, die effektiv abgewandert waren und in der Zwischenzeit wieder nach Italien zurückgekehrt sind, werden ersucht, ihre Erklärungen zwecks Wiedererlangung der italienischen Staatsbürger- schaft erst nach dem 20. Februar 1948 abzu geben, um das Optionsamt im gegenwärtigen Zeitpunkt nicht zu überlasten. Die eingebürgerten Optanten, die in Italien verblieben sind, sowie alle diejenigen, welche nach Art. 15 des Optionsdekretes nicht als „abgewandert“ anzusehen sind, werden dringend aufgefordert, ihre Erklärungen nach Art. 2 des erwähnten Optionsdekretes rechtzeitig. d. i. bis zum 20, Februar 1948, einzureichen, und zwar auch dann, wenn sie in den Wählerlisten eingetragen sind, weil sie sonst Gefahr laufen, aus denselben gestrichen zu werden. Es wird ausdrücklich darauf aufmerksam gemacht, daß viele eingebürgerte Optanten die auf Grund der gesetzlichen Bestimmungen nicht als abgewandert zu gelten haben, sowie auch solche, die während der ganzen Zeit Italien überhaupt nie verlassen haben, in den Refistern der Gemeinden als abgewandert figurieren; diese werden daher besonders gebeten, ihre Erklärunden fristgerecht, d. i. bis zum 20. Februar 1948, abzugeben, und falls die Eintragung in die Wählerlisten verweigert werden sollte, umgehend bei der Bezirkskommission (Commissione Elettorale Mandamentale) Einspruch zu erheben. Zu diesem Zwecke ist es erforderlich, daß jeder eingebürgerte Optant, der die obenerwähnte Erklärung abgegeben hat, sich persönlich davon überzeugt, ob die Eintragung in die Wählerlisten erfolgt ist. Die sogenannten Rückwähler, das sind diejenigen, welche vor dem 9. September 1943 bei der Präfektur um Annullierung der Option für Deutschland angesucht haben, müssen sich persönlich davon überzeugen, ob sie auf Grund der Rückoption auch tatsächlich wieder als italienische Staatsbürger eingetragen wurden. Ist dies nicht der Fall oder besieht diesbezüglich irgend eine Unsicherheit, so müssen die selben die im Optionsgesetz vorgesehene Erklärung abdeben, weil sonst die Gelahr besteht, daß sie die italienische Staatsbürgerschaft nicht erlangen. Die Optanten werden aufmerksam gemacht, daß die Präfektur Bozen an alle Gemeinden die Weisung ergehen ließ, bei der Ausgabe der Erklärungsformulare keinerlei Schwierigkeiten zu bereiten, weshalb alle Optanten denen die Ausfolgung der Formulare verweigert wurde ersucht werden, dieselben neuerlich anzufordern.","['achtung', 'südtiroler', 'optanten', 'deutschland', 'effektiv', 'abgewandert', 'zwischenzeit', 'italien', 'zurückgekehrt', 'ersucht', 'erklärungen', 'zwecks', 'wiedererlangung', 'italienischen', 'staatsbürger', 'schaft', 'erst', '20', 'februar', '1948', 'abzu', 'geben', 'optionsamt', 'gegenwärtigen', 'zeitpunkt', 'überlasten', 'eingebürgerten', 'optanten', 'italien', 'verblieben', 'sowie', 'diejenigen', 'art', '15', 'optionsdekretes', 'abgewandert', 'anzusehen', 'dringend', 'aufgefordert', 'erklärungen', 'art', '2', 'erwähnten', 'optionsdekretes', 'rechtzeitig', 'd', 'i', '20', 'februar', '1948', 'einzureichen', 'wählerlisten', 'eingetragen', 'gefahr', 'laufen', 'gestrichen', 'ausdrücklich', 'darauf', 'aufmerksam', 'gemacht', 'viele', 'eingebürgerte', 'optanten', 'grund', 'gesetzlichen', 'bestimmungen', 'abgewandert', 'gelten', 'sowie', 'ganzen', 'zeit', 'italien', 'überhaupt', 'nie', 'verlassen', 'refistern', 'gemeinden', 'abgewandert', 'figurieren', 'daher', 'besonders', 'gebeten', 'erklärunden', 'fristgerecht', 'd', 'i', '20', 'februar', '1948', 'abzugeben', 'falls', 'eintragung', 'wählerlisten', 'verweigert', 'umgehend', 'bezirkskommission', 'commissione', 'elettorale', 'mandamentale', 'einspruch', 'erheben', 'zwecke', 'erforderlich', 'eingebürgerte', 'optant', 'obenerwähnte', 'erklärung', 'abgegeben', 'persönlich', 'davon', 'überzeugt', 'eintragung', 'wählerlisten', 'erfolgt', 'sogenannten', 'rückwähler', 'diejenigen', '9', '

## Named Emtities Recognition <a class="anchor" id="6-bullet"></a>

In [6]:
from spacy import displacy
NER = spacy.load('de_core_news_sm')
data_ready2 = NER(df['text'][0])
text = displacy.render(data_ready2,style="ent",jupyter=True)
print(text)

None


In [7]:
for i in range(len(df)) :
    df.iloc[i,6] = ' '.join(df.iloc[i,6])
nlp = spacy.load('de_core_news_sm')
doc = nlp(df['lemmatized'][0])
for ent in doc.ents:
    print(ent.text, ent.label_)  

deutschland LOC
italien LOC
italien LOC
italien LOC
deutschland LOC
italienische MISC
italienische MISC
präfektur bozen LOC


## Gewünschten Text exportieren <a class="anchor" id="7-bullet"></a>

Am Ende können Sie die gewünschten Resultate in Form der Excel Tabelle exportieren. 

In [8]:

deselectlist =[ 'tokenized', 'tagging']
selectlist =[x for x in df.columns if x not in deselectlist]
df = df[selectlist]
    
df.to_excel('results.xlsx')
df

,jahrgang,zeitung,datum,ausgabe,text,lemmatized
0,1946,Dolomiten,1946-11-10,2,Achtung! Südtiroler!\n\n\nDie Optanten für Deu...,achtung südtiroler optanten deutschland effekt...
1,1947,Dolomiten,1947-11-10,3,"\nDabei muß klargestellt werden, daß der\njetz...",dabei muß klarstellen jetzig belegschaft wiede...
